<a href="https://colab.research.google.com/github/Yuming293/personal1/blob/main/SDW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title 👇 首次运行！
import os
import sys
import binascii
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets

sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w = binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')
sai = binascii.a2b_uu("=<W1A8FQE+61I9F9U<VEO;BUS=&%B:6QI='DM86D`").decode('utf-8')
sd = binascii.a2b_uu("04W1A8FQE+61I9F9U<VEO;@``").decode('utf-8')

sd_dir = '/root/main'
gsd_dir = f'/root/drive/MyDrive/main'

#Check GPU  ,不使用GPU会出现莫名其妙的错误哦。
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
if tf.test.gpu_device_name():
    print("GPU is available")
else:
    print("GPU is NOT available")
    raise Exception("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")

# git clone AUTOMATIC1111
!apt -y install -qq aria2 git
%cd /root   
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  !git clone -q --branch master https://github.com/AUTOMATIC1111/$sdw /root/main
  #!git clone -q --branch v2.6 https://github.com/camenduru//$sdw /root/main

# Requirements 
with capture.capture_output() as cap:
  %cd /root
  !mkdir -p {sd_dir}/repositories/{sai}
  !wget -q -i https://raw.githubusercontent.com/Van-wise/fast-sd/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  !tar -C /root/ --zstd -xf sd_mrep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  !mv -n /root/sd/stablediffusion/* {sd_dir}/repositories/{sai} && rm -rf /root/sd
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py

!apt --fix-broken install
!pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv image-reward pyfunctional pycloudflared pyngrok torch PyExecJS pathos boto3
!sudo apt-get update
!sudo apt-get install aria2

clear_output()
inf('\u2714 Done','success', '50px')

#负面提示词
!git clone https://huggingface.co/embed/negative /root/negative
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /root/negative/* {sd_dir}/embeddings && rm -rf /root/negative
!git clone https://huggingface.co/embed/lora /root/lora
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /root/lora/* {sd_dir}/models/Lora && rm -rf /root/lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {sd_dir}/models/ESRGAN -o 4x-UltraSharp.pth

#同步 秋葉aaaki Webui (可选)
%cd {sd_dir}
config_link = "https://raw.githubusercontent.com/Van-wise/sd-colab/main/config.json" #@param {type:"string"}"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {config_link} -d {sd_dir} -o config.json

clear_output()
inf('\u2714 Done','success', '50px')


#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {sd_dir}/modules
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  
    %cd {sd_dir}
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {sd_dir}/webui.py
    !sed -i 's@print(\"No module.*@@' {sd_dir}/repositories/{sai}/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {sd_dir}/modules/shared.py

    !sed -i 's@possible_sd_paths =.*@possible_sd_paths = [\"{sd_dir}/repositories/{sai}\"]@' {sd_dir}/modules/paths.py
    !sed -i 's@\.\.\/@src/@g' {sd_dir}/modules/paths.py
    !sed -i 's@src/generative-models@generative-models@g' {sd_dir}/modules/paths.py
    
!curl -Lo memfix.zip https://github.com/nolanaatama/microsoftexcel/raw/main/memfix.zip
!unzip /root/memfix.zip
!apt install -qq libunwind8-dev
!apt install -qq libcairo2-dev pkg-config python3-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *
!pip install fastapi==0.90
!curl -Lo microsoftexcel.zip https://huggingface.co/nolanaatama/colab/resolve/main/microsoftexcel151.zip
!unzip /root/microsoftexcel.zip


#wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers --disable-console-progressbars --theme dark --upcast-sampling"
#!python {sd_dir}/webui.py $wise


In [0]:
#链接 controlnet
#@markdown 选择ControlNet：
#@markdown 
#@markdown 0：不需要使用ControlNet
#@markdown 
#@markdown 1：下载ControlNet 模型到 Colab
controlnet = 1 #@param {type:"slider", min:0, max:1, step:1}
if controlnet == 1:
    # 下载ControlNet 模型到 Colab
    #ControlNet Model Download 
    !wget -q -O cnmodels.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/cnmodels.py
    from cnmodels import download
    from cnmodels import cndown_colab
    cndown_colab()
else:
    print("不使用 ControlNet、或已使用快捷方式。")

In [0]:
# @title 下载其他必备文件

from IPython.display import clear_output

!wget https://github.com/BBrother-GFC/SDW/raw/main/BianLiang.txt -O /root/main/BianLiang.txt
!wget https://github.com/BBrother-GFC/SDW/raw/main/BuShu.py -O /root/main/BuShu.py
!wget https://github.com/BBrother-GFC/SDW/raw/main/nologs.py -O /root/main/nologs.py
!wget https://github.com/BBrother-GFC/SDW/raw/main/Authtoken.txt -O /root/main/Authtoken.txt
!wget https://github.com/BBrother-GFC/SDW/raw/main/webui.py -O /root/main/webui.py
!wget https://github.com/BBrother-GFC/SDW/raw/main/SW_run.py -O /root/main/SW_run.py
!wget https://github.com/BBrother-GFC/SDW/raw/main/StartBianLiang.py -O /root/main/StartBianLiang.py
!wget https://github.com/BBrother-GFC/SDW/raw/main/launch.py -O /root/main/launch.py
!wget https://gist.githubusercontent.com/journey-ad/d98ed173321658be6e51f752d6e6163c/raw/aa162c81d9d7d7b6efdffa84ccbfe867be6b711d/I18N_sd-webui-zh_CN.json -O /root/main/localizations/zh_CN.json

clear_output()
inf('\u2714 Done','success', '50px')

In [0]:
'''
Author: BBrother-GFC 3347951573@qq.com
Date: 2023-10-01 11:27:10
LastEditors: BBrother-GFC 3347951573@qq.com
LastEditTime: 2023-10-01 12:03:22
FilePath: \SDW-main\SDW_v6_2.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
#@title _Ngrok与ngrokToken_
#StartBL = '--skip-torch-cuda-test', '--xformers', '--enable-insecure-extension-access', '--gradio-queue', '--disable-nan-check', '--enable-console-prompts', '--opt-split-attention', '--disable-safe-unpickle', '--theme', 'dark', '--deforum-api' #@param {type:"string"}
ngrok_use = False #@param {type:"boolean"}
ngrokToken = "" #@param {type:"string"}

# 追加到文件
with open("/root/main/StartBianLiang.txt", "w+") as file:
    #file.write(f"StartBL = {StartBL}\n")

    file.write(f"ngrok_use = {ngrok_use}\n")
with open("/root/main/Authtoken.txt", "w+") as file:
    file.write(f"{ngrokToken}")
inf('\u2714 Done','success', '50px')


In [0]:
#@title Checkpoints/safetensors等大模型下载
#@markdown - _下载任务1_
CS_download1 = False #@param {type:"boolean"}
CS_url1 = "https://civitai.com/api/download/models/17701" #@param {type:"string"}
CS_filename1 = "anypastelAnythingV45_anypastelAnythingV45.safetensors" #@param {type:"string"}

#@markdown - _下载任务2_
CS_download2 = False #@param {type:"boolean"}
CS_url2 = "https://civitai.com/api/download/models/26633?type=Model&format=PickleTensor&size=full&fp=fp16" #@param {type:"string"}
CS_filename2 = "[\u4E8C\u6B21\u5143]07CounterfeitV2503_10.ckpt" #@param {type:"string"}

#@markdown - _下载任务3_
CS_download3 = False #@param {type:"boolean"}
CS_url3 = "https://civitai.com/api/download/models/90854" #@param {type:"string"}
CS_filename3 = "[\u4E8C\u6B21\u5143]AnythingV5Ink.safetensors" #@param {type:"string"}

#@markdown - _下载任务4_
CS_download4 = False #@param {type:"boolean"}
CS_url4 = "https://civitai.com/api/download/models/57618" #@param {type:"string"}
CS_filename4 = "[\u4E8C\u6B21\u5143]CounterfeitV30_v30.safetensors" #@param {type:"string"}

#@markdown - _下载任务5_
CS_download5 = False #@param {type:"boolean"}
CS_url5 = "https://civitai.com/api/download/models/1356" #@param {type:"string"}
CS_filename5 = "dreamlike10_10.ckpt" #@param {type:"string"}

#@markdown - _下载任务6_
CS_download6 = False #@param {type:"boolean"}
CS_url6 = "https://civitai.com/api/download/models/17233" #@param {type:"string"}
CS_filename6 = "[\u4E8C\u6B21\u5143]abyssorangemix3AOM3_aom3a1b.safetensors" #@param {type:"string"}

#@markdown - _下载任务7_
CS_download7 = True #@param {type:"boolean"}
CS_url7 = "https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors" #@param {type:"string"}
CS_filename7 = "chilloutmix_NiPrunedFp32Fix.safetensors" #@param {type:"string"}

#@markdown - _下载任务8_
CS_download8 = False #@param {type:"boolean"}
CS_url8 = "" #@param {type:"string"}
CS_filename8 = "" #@param {type:"string"}

#@markdown - _下载任务9_
CS_download9 = False #@param {type:"boolean"}
CS_url9 = "" #@param {type:"string"}
CS_filename9 = "" #@param {type:"string"}

#@markdown - _下载任务10_
CS_download10 = False #@param {type:"boolean"}
CS_url10 = "" #@param {type:"string"}
CS_filename10 = "" #@param {type:"string"}

#@markdown - _下载任务11_
CS_download11 = False #@param {type:"boolean"}
CS_url11 = "" #@param {type:"string"}
CS_filename11 = "" #@param {type:"string"}

#@markdown - _下载任务12_
CS_download12 = False #@param {type:"boolean"}
CS_url12 = "" #@param {type:"string"}
CS_filename12 = "" #@param {type:"string"}

#@markdown - _下载任务13_
CS_download13 = False #@param {type:"boolean"}
CS_url13 = "" #@param {type:"string"}
CS_filename13 = "" #@param {type:"string"}

# 追加到文件
with open("/root/main/BianLiang.txt", "a") as file:
    file.write(f"CS_download1 = {CS_download1}\n")
    file.write(f"CS_url1 = '{CS_url1}'\n")
    file.write(f"CS_filename1 = '{CS_filename1}'\n")

    file.write(f"CS_download2 = {CS_download2}\n")
    file.write(f"CS_url2 = '{CS_url2}'\n")
    file.write(f"CS_filename2 = '{CS_filename2}'\n")

    file.write(f"CS_download3 = {CS_download3}\n")
    file.write(f"CS_url3 = '{CS_url3}'\n")
    file.write(f"CS_filename3 = '{CS_filename3}'\n")

    file.write(f"CS_download4 = {CS_download4}\n")
    file.write(f"CS_url4 = '{CS_url4}'\n")
    file.write(f"CS_filename4 = '{CS_filename4}'\n")

    file.write(f"CS_download5 = {CS_download5}\n")
    file.write(f"CS_url5 = '{CS_url5}'\n")
    file.write(f"CS_filename5 = '{CS_filename5}'\n")

    file.write(f"CS_download6 = {CS_download6}\n")
    file.write(f"CS_url6 = '{CS_url6}'\n")
    file.write(f"CS_filename6 = '{CS_filename6}'\n")

    file.write(f"CS_download7 = {CS_download7}\n")
    file.write(f"CS_url7 = '{CS_url7}'\n")
    file.write(f"CS_filename7 = '{CS_filename7}'\n")

    file.write(f"CS_download8 = {CS_download8}\n")
    file.write(f"CS_url8 = '{CS_url8}'\n")
    file.write(f"CS_filename8 = '{CS_filename8}'\n")

    file.write(f"CS_download9 = {CS_download9}\n")
    file.write(f"CS_url9 = '{CS_url9}'\n")
    file.write(f"CS_filename9 = '{CS_filename9}'\n")

    file.write(f"CS_download10 = {CS_download10}\n")
    file.write(f"CS_url10 = '{CS_url10}'\n")
    file.write(f"CS_filename10 = '{CS_filename10}'\n")

    file.write(f"CS_download11 = {CS_download11}\n")
    file.write(f"CS_url11 = '{CS_url11}'\n")
    file.write(f"CS_filename11 = '{CS_filename11}'\n")

    file.write(f"CS_download12 = {CS_download12}\n")
    file.write(f"CS_url12 = '{CS_url12}'\n")
    file.write(f"CS_filename12 = '{CS_filename12}'\n")

    file.write(f"CS_download13 = {CS_download13}\n")
    file.write(f"CS_url13 = '{CS_url13}'\n")
    file.write(f"CS_filename13 = '{CS_filename13}'\n")

inf('\u2714 Done','success', '50px')


In [0]:
#@title Lora模型下载

#@markdown - _下载任务1_
Lora_download = False #@param {type:"boolean"}
Lora_url1 = "https://civitai.com/api/download/models/64842" #@param {type:"string"}
Lora_filename = "arona_puranalora_v1.safetensors" #@param {type:"string"}

#@markdown - _下载任务2_
Lora_download2 = False #@param {type:"boolean"}
Lora_url2 = "https://liblibai-online.liblibai.ai/web/model/25e9f1fa5137b2517b46c0a07f068e7328d5bd28744255d476c40e3ce61abed4.safetensors?attname=Xian-T%E6%89%8B%E9%83%A8%E4%BF%AE%E5%A4%8Dlora%EF%BC%88%E4%B8%8D%E7%94%A8controlnet%E4%B9%9F%E4%B8%8D%E5%9D%8F%E6%89%8B%E4%BA%86%EF%BC%89_v3.0.safetensors" #@param {type:"string"}
Lora_filename2 = "Xian-T\u624B\u90E8\u4FEE\u590Dlora\uFF08%E4%B8%8D%E7%94%A8controlnet%E4%B9%9F%E4%B8%8D%E5%9D%8F%E6%89%8B%E4%BA%86%E4%BB%80%E4%B9%88%EF%BC%89_v3.0.safetensors" #@param {type:"string"}

#@markdown - _下载任务3_
Lora_download3 = False #@param {type:"boolean"}
Lora_url3 = "https://civitai.com/api/download/models/52900" #@param {type:"string"}
Lora_filename3 = "cailaiarona-000007.safetensors" #@param {type:"string"}

#@markdown - _下载任务4_
Lora_download4 = True #@param {type:"boolean"}
Lora_url4 = "https://civitai.com/api/download/models/9409" #@param {type:"string"}
Lora_filename4 = "chara-arona-v1.safetensors" #@param {type:"string"}

#@markdown - _下载任务5_
Lora_download5 = False #@param {type:"boolean"}
Lora_url5 = "" #@param {type:"string"}
Lora_filename5 = "" #@param {type:"string"}

#@markdown - _下载任务6_
Lora_download6 = False #@param {type:"boolean"}
Lora_url6 = "" #@param {type:"string"}
Lora_filename6 = "" #@param {type:"string"}

# 写入到文件
with open("/root/main/BianLiang.txt", "a") as file:
    file.write(f"Lora_download = {Lora_download}\n")
    file.write(f"Lora_url1 = '{Lora_url1}'\n")
    file.write(f"Lora_filename = '{Lora_filename}'\n")

    file.write(f"Lora_download2 = {Lora_download2}\n")
    file.write(f"Lora_url2 = '{Lora_url2}'\n")
    file.write(f"Lora_filename2 = '{Lora_filename2}'\n")

    file.write(f"Lora_download3 = {Lora_download3}\n")
    file.write(f"Lora_url3 = '{Lora_url3}'\n")
    file.write(f"Lora_filename3 = '{Lora_filename3}'\n")

    file.write(f"Lora_download4 = {Lora_download4}\n")
    file.write(f"Lora_url4 = '{Lora_url4}'\n")
    file.write(f"Lora_filename4 = '{Lora_filename4}'\n")

    file.write(f"Lora_download5 = {Lora_download5}\n")
    file.write(f"Lora_url5 = '{Lora_url5}'\n")
    file.write(f"Lora_filename5 = '{Lora_filename5}'\n")

    file.write(f"Lora_download6 = {Lora_download6}\n")
    file.write(f"Lora_url6 = '{Lora_url6}'\n")
    file.write(f"Lora_filename6 = '{Lora_filename6}'\n")

inf('\u2714 Done','success', '50px')


In [0]:
#@title VAE下载
#@markdown - _下载任务1_
VAE_download1 = False #@param {type:"boolean"}
VAE_url1 = "https://civitai.com/api/download/models/25986" #@param {type:"string"}
VAE_filename1 = "anypastelAnythingV45_anypastelAnythingV45.zip" #@param {type:"string"}

#@markdown - _下载任务2_
VAE_download2 = False #@param {type:"boolean"}
VAE_url2 = "https://civitai.com/api/download/models/64133" #@param {type:"string"}
VAE_filename2 = "counterfeit.vae.pt" #@param {type:"string"}

#@markdown - _下载任务3_
VAE_download3 = True #@param {type:"boolean"}
VAE_url3 = "https://civitai.com/api/download/models/80869?type=Model&format=Other" #@param {type:"string"}
VAE_filename3 = "vaeFtMse840000Ema_v100.safetensors" #@param {type:"string"}

#@markdown - _下载任务4_
VAE_download4 = False #@param {type:"boolean"}
VAE_url4 = "https://huggingface.co/admruul/anything-v3.0/resolve/main/Anything-V3.0.vae.pt" #@param {type:"string"}
VAE_filename4 = "Anything-V3.0.vae.pt" #@param {type:"string"}

#@markdown - _下载任务5_
VAE_download5 = False #@param {type:"boolean"}
VAE_url5 = "https://civitai.com/api/download/models/131654?type=Model&format=SafeTensor" #@param {type:"string"}
VAE_filename5 = "anythingKlF8Anime2VaeFtMse840000_klF8Anime2.safetensors" #@param {type:"string"}

#@markdown - _下载任务6_
VAE_download6 = False #@param {type:"boolean"}
VAE_url6 = "https://civitai.com/api/download/models/88156?type=Model&format=Other" #@param {type:"string"}
VAE_filename6 = "ClearVAE.2.3.vae.pt" #@param {type:"string"}

#@markdown - _下载任务7_
VAE_download7 = False #@param {type:"boolean"}
VAE_url7 = "https://civitai.com/api/download/models/121354?type=Model&format=Other" #@param {type:"string"}
VAE_filename7 = "orangemix_v1.pt" #@param {type:"string"}

#@markdown - _下载任务8_
VAE_download8 = False #@param {type:"boolean"}
VAE_url8 = "https://civitai.com/api/download/models/28569?type=Model&format=Other" #@param {type:"string"}
VAE_filename8 = "klF8Anime2_klF8Anime2VAE.pt" #@param {type:"string"}

#@markdown - _下载任务9_
VAE_download9 = False #@param {type:"boolean"}
VAE_url9 = "" #@param {type:"string"}
VAE_filename9 = "" #@param {type:"string"}

#@markdown - _下载任务10_
VAE_download10 = False #@param {type:"boolean"}
VAE_url10 = "" #@param {type:"string"}
VAE_filename10 = "" #@param {type:"string"}

#@markdown - _下载任务11_
VAE_download11 = False #@param {type:"boolean"}
VAE_url11 = "" #@param {type:"string"}
VAE_filename11 = "" #@param {type:"string"}

#@markdown - _下载任务12_
VAE_download12 = False #@param {type:"boolean"}
VAE_url12 = "" #@param {type:"string"}
VAE_filename12 = "" #@param {type:"string"}

#@markdown - _下载任务13_
VAE_download13 = False #@param {type:"boolean"}
VAE_url13 = "" #@param {type:"string"}
VAE_filename13 = "" #@param {type:"string"}

# 写入到文件
with open("/root/main/BianLiang.txt", "a") as file:
    file.write(f"VAE_download1 = {VAE_download1}\n")
    file.write(f"VAE_url1 = '{VAE_url1}'\n")
    file.write(f"VAE_filename1 = '{VAE_filename1}'\n")

    file.write(f"VAE_download2 = {VAE_download2}\n")
    file.write(f"VAE_url2 = '{VAE_url2}'\n")
    file.write(f"VAE_filename2 = '{VAE_filename2}'\n")

    file.write(f"VAE_download3 = {VAE_download3}\n")
    file.write(f"VAE_url3 = '{VAE_url3}'\n")
    file.write(f"VAE_filename3 = '{VAE_filename3}'\n")

    file.write(f"VAE_download4 = {VAE_download4}\n")
    file.write(f"VAE_url4 = '{VAE_url4}'\n")
    file.write(f"VAE_filename4 = '{VAE_filename4}'\n")

    file.write(f"VAE_download5 = {VAE_download5}\n")
    file.write(f"VAE_url5 = '{VAE_url5}'\n")
    file.write(f"VAE_filename5 = '{VAE_filename5}'\n")

    file.write(f"VAE_download6 = {VAE_download6}\n")
    file.write(f"VAE_url6 = '{VAE_url6}'\n")
    file.write(f"VAE_filename6 = '{VAE_filename6}'\n")

    file.write(f"VAE_download7 = {VAE_download7}\n")
    file.write(f"VAE_url7 = '{VAE_url7}'\n")
    file.write(f"VAE_filename7 = '{VAE_filename7}'\n")

    file.write(f"VAE_download8 = {VAE_download8}\n")
    file.write(f"VAE_url8 = '{VAE_url8}'\n")
    file.write(f"VAE_filename8 = '{VAE_filename8}'\n")

    file.write(f"VAE_download9 = {VAE_download9}\n")
    file.write(f"VAE_url9 = '{VAE_url9}'\n")
    file.write(f"VAE_filename9 = '{VAE_filename9}'\n")

    file.write(f"VAE_download10 = {VAE_download10}\n")
    file.write(f"VAE_url10 = '{VAE_url10}'\n")
    file.write(f"VAE_filename10 = '{VAE_filename10}'\n")

    file.write(f"VAE_download11 = {VAE_download11}\n")
    file.write(f"VAE_url11 = '{VAE_url11}'\n")
    file.write(f"VAE_filename11 = '{VAE_filename11}'\n")

    file.write(f"VAE_download12 = {VAE_download12}\n")
    file.write(f"VAE_url12 = '{VAE_url12}'\n")
    file.write(f"VAE_filename12 = '{VAE_filename12}'\n")

    file.write(f"VAE_download13 = {VAE_download13}\n")
    file.write(f"VAE_url13 = '{VAE_url13}'\n")
    file.write(f"VAE_filename13 = '{VAE_filename13}'\n")

inf('\u2714 Done','success', '50px')


In [0]:
#@title embeddings模型下载

#@markdown - _下载任务1_
embeddings_download1 = True #@param {type:"boolean"}
embeddings_url1 = "https://civitai.com/api/download/models/125849" #@param {type:"string"}
embeddings_filename1 = "bad-hands-5.pt" #@param {type:"string"}

# 任务2
#@markdown - _下载任务2_
embeddings_download2 = False #@param {type:"boolean"}
embeddings_url2 = "" #@param {type:"string"}
embeddings_filename2 = "" #@param {type:"string"}

# 任务3
#@markdown - _下载任务3_
embeddings_download3 = False #@param {type:"boolean"}
embeddings_url3 = "" #@param {type:"string"}
embeddings_filename3 = "" #@param {type:"string"}

# 任务4
#@markdown - _下载任务4_
embeddings_download4 = False #@param {type:"boolean"}
embeddings_url4 = "" #@param {type:"string"}
embeddings_filename4 = "" #@param {type:"string"}

# 任务5
#@markdown - _下载任务5_
embeddings_download5 = False #@param {type:"boolean"}
embeddings_url5 = "" #@param {type:"string"}
embeddings_filename5 = "" #@param {type:"string"}

# 写入到文件
with open("/root/main/BianLiang.txt", "a") as file:
    file.write(f"embeddings_download1 = {embeddings_download1}\n")
    file.write(f"embeddings_url1 = '{embeddings_url1}'\n")
    file.write(f"embeddings_filename1 = '{embeddings_filename1}'\n")

    file.write(f"embeddings_download2 = {embeddings_download2}\n")
    file.write(f"embeddings_url2 = '{embeddings_url2}'\n")
    file.write(f"embeddings_filename2 = '{embeddings_filename2}'\n")

    file.write(f"embeddings_download3 = {embeddings_download3}\n")
    file.write(f"embeddings_url3 = '{embeddings_url3}'\n")
    file.write(f"embeddings_filename3 = '{embeddings_filename3}'\n")

    file.write(f"embeddings_download4 = {embeddings_download4}\n")
    file.write(f"embeddings_url4 = '{embeddings_url4}'\n")
    file.write(f"embeddings_filename4 = '{embeddings_filename4}'\n")

    file.write(f"embeddings_download5 = {embeddings_download5}\n")
    file.write(f"embeddings_url5 = '{embeddings_url5}'\n")
    file.write(f"embeddings_filename5 = '{embeddings_filename5}'\n")

inf('\u2714 Done','success', '50px')